In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.quantum_info import Operator

from qiskit.tools.visualization import plot_histogram
from IPython.display import display, Math, Latex

import pennylane as qml

# Variational Quantum Eigensolver

A well known problem from chemistry to computer science consists into finding the minimum eigenvalue of a matrix. In chemistry this problem can be characterized into finding the ground state of a system defined by a molecule. To estimate these values we can use a variational method called Variational Quantum Eigensolver (VQE).

The VQE is an application of the Ritz Variational Method, where a Quantum Computer prepare a Ansatz trial state given by a wave function $\Psi$, within boundary conditions and adjustable parameters the may be used to find the lowest energy of a quantum system (minimum eigenvalue), given by,

$$
    E_0 \leq \frac{\langle\psi|H|\psi\rangle}{\langle\psi|\psi\rangle}
$$
## Mathematical Basics

Let $M$ be a symmetric matrix, we say that $v$ is a eigenvector and $\lambda$ a eigenvalue of $M$ if the following equation holds

$$
  Av = \lambda v
$$

Moreover, let $H$ be our hamiltonian hermitian $nxn$ matrix, and $|\psi\rangle$ one of its eigenvectors, then

$$
  H|\psi\rangle = \lambda |\psi\rangle
$$

And by the spectral theorem, we may rewrite this matrix in function of its eigenvectors and eigenvalues such that

$$
    H = \sum_{i = 0}^{n-1}\lambda_i|\psi_i\rangle\langle\psi_i|
$$

Where the eigenvalue $\lambda_i$ is associated with the eigenvector $|\psi_i\rangle$ and $|\psi_i\rangle\langle\psi_i|$ are projections onto its eigenspaces, and by the theorem as $H$ is hermitian, then for all $\lambda_i$, $\lambda_i = \lambda_i^*$. Futhermore, from that we may extract that the expectation value of the observable $H$ on an arbitrary quantum state $|\psi\rangle$ is given by

$$
    \langle H \rangle_\psi = \langle\psi|H|\psi\rangle
$$

Then, by rewriting the last equation by the spetral decomposition of $H$, we get that 

$$
    \begin{align*}
    \langle H \rangle_\psi &= \langle\psi|H|\psi\rangle\\
                           &= \langle\psi|\bigg(\sum_{i = 0}^{n-1}\lambda_i|\psi_i\rangle\langle\psi_i|\bigg)|\psi\rangle\\
                           &=\sum_{i = 0}^{n-1}\lambda_i\langle\psi||\psi_i\rangle\langle\psi_i||\psi\rangle\\
                           &=\sum_{i = 0}^{n-1}\lambda_i|\langle\psi||\psi_i\rangle|^2\\
    \end{align*}
$$

From this last equation get that our observable on any state will be given by a linear combination of $|\langle\psi||\psi_i\rangle|^2 \geq 0$ up to a scalar $\lambda_i$. Moreover, we have that $\lambda_0$ is bounded by our measured result

$$
    \lambda_{0} \leq \langle H \rangle_\psi = \langle\psi|H|\psi\rangle = \sum_{i = 0}^{n-1}\lambda_i|\langle\psi||\psi_i\rangle|^2
$$

Furthermore if we take $|\psi_0\rangle$, which is the eigenvector associated with the lowest energy of our quantum system, as all of our eigenvectors are orthornormal with each other, then equality holds for our last equation and $\lambda_{0} = \langle H \rangle_{\psi_0}$.

Now, we need to define our variational parameters, to approximate the required Ansatz trial states. So let $U(\theta)$ be a unitary operator in function of a given angle $\theta$, then applying this linear transformation onto a state $|\psi\rangle$ we get that

$$
    U(\theta)|\psi\rangle = |\psi(\theta)\rangle
$$

And our goal is to optimize our parameters in order to get an approximate solution to our minimum eigenvalue problem, so that 

$$
\langle H \rangle_{\psi(\theta)} = \langle\psi(\theta)|H|\psi(\theta)\rangle \geq E_0 = \lambda_0
$$

## Pennylane Implementation

We can implement a simple single qubit version of the VQE using the Pennylane library as follows

In [2]:
#Define our backend and the number of qubits
backend = qml.device("default.qubit", wires=1)

#Define the unitary variational circuit by applying RX(\theta_0) and RY(\theta_1)
@qml.qnode(backend)
def circuit(angles):
    qml.RX(angles[0], wires = 0)
    qml.RY(angles[1], wires = 0)
    
    return qml.expval(qml.PauliZ(0))

In [3]:
#Set the parameters optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.6)

#Define a cost function for our circuit
def cost(param):
    return circuit(param)

In [4]:
#Set appropriate initial parameters
initi_params = np.array([0.687, 0.258])

#Set the number of optimization layers to be applied
layers = 20

for i in range(layers):
    initi_params = opt.step(cost, initi_params)
    print("Expectation Ground State Energy Value {:3d}: {: .7f}".format(i, cost(initi_params)))

print("Optimized Parameters: {}".format(initi_params))

Expectation Ground State Energy Value   0:  0.4587564
Expectation Ground State Energy Value   1:  0.0269474
Expectation Ground State Energy Value   2: -0.4222236
Expectation Ground State Energy Value   3: -0.7694359
Expectation Ground State Energy Value   4: -0.9428985
Expectation Ground State Energy Value   5: -0.9897271
Expectation Ground State Energy Value   6: -0.9983205
Expectation Ground State Energy Value   7: -0.9997303
Expectation Ground State Energy Value   8: -0.9999568
Expectation Ground State Energy Value   9: -0.9999931
Expectation Ground State Energy Value  10: -0.9999989
Expectation Ground State Energy Value  11: -0.9999998
Expectation Ground State Energy Value  12: -1.0000000
Expectation Ground State Energy Value  13: -1.0000000
Expectation Ground State Energy Value  14: -1.0000000
Expectation Ground State Energy Value  15: -1.0000000
Expectation Ground State Energy Value  16: -1.0000000
Expectation Ground State Energy Value  17: -1.0000000
Expectation Ground State Ene

### References

[[1]](https://pennylane.ai/qml/demos/tutorial_qubit_rotation.html) PennyLane Basic tutorial: qubit rotation

[[2]](https://pennylane.ai/qml/demos/tutorial_vqe.html) PennyLane A brief overview of VQE

[[3]](https://qiskit.org/textbook/ch-applications/vqe-molecules.html) Qiskit Simulating Molecules using VQE